In [191]:
import pandas as pd
import numpy as np
import requests
import unicodedata
import re
from fuzzywuzzy import fuzz, process
from bs4 import BeautifulSoup

url = 'https://ideas.repec.org/e/poa5.html'

# Setup beautiful soup
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

# Publication classifications free, gate, none
# There will be overlaps
publications = soup.find_all('li', class_={'list-group-item downfree', \
    'list-group-item downgate', 'list-group-item downnone'})

In [193]:
paper_details = {}
i = 1
for pub in publications:
    try:
        title = pub.find('a').text
        name_year = pub.text.strip().split('\n')[0]
        

        if 'undated' in name_year:
            year = 'undated'
            authors = re.sub(r', \"undated\"', '',name_year).split(' & ')
        else:
            year = int(re.findall(r', (\d{4})\.', name_year)[0])
            authors = re.sub(r', \d{4}\.', '',name_year).split(' & ')
        
        paper_details[title] = {'url':url, 'author': authors, 'year': year}
    except:
        print('something went wrong at paper {}'.format(i))
    i +=1
    

In [197]:
personal_details = soup.find('tbody').find_all('tr')

In [199]:
personal_details[0]

<tr>
<td>First Name:</td><td>Bogdan</td>
</tr>

In [205]:
per_clean

{'First Name': 'Bogdan',
 'Last Name': 'Oancea',
 'RePEc Short-ID': 'poa5',
 'Twitter': '@bogdan__oancea'}

In [219]:
personal_details = soup.find('tbody').find_all('tr')
per = {}
for p in personal_details:
    k = p.find_all('td')[0].text.replace(':','')
    v = p.find_all('td')[1].text.strip()
    per[k] = v
per_clean = {k:v for (k,v) in per.items() if (v is not '') }

# Find homepage link
try:    
    homepage = soup.find('td', {'class':'homelabel'}).next_sibling.find('a', href=True)['href']
    per_clean['Homepage'] = homepage
except:
    print('homepage not found')

# Find affiliation - can have multiple
affiliation_soup = soup.find('div', {'id':'affiliation'})
i = 0
try:
    for a in affiliation_soup.find_all('h3'):
        if a.find('br'):
            department = a.find('br').previous_sibling
            organisation = a.find('br').next_sibling
        else:
            print('no breaks in affiliation')
            department = ''
            organisation = a
            .text
        per_clean['Aff_Department{}'.format(i)] = department
        per_clean['Aff_Organisation{}'.format(i)] = organisation
        i += 1
except:
    print('affiliation not found')

# Find affiliation locations
i = 0
try:
    for a in affiliation_soup.find_all('span', {'class':'locationlabel'}):
        if a:
            location = a.text
        else:
            print('no location in affiliation')
        per_clean['Aff_Location{}'.format(i)] = location
        i += 1
except:
    print('affiliation not found')
print(per_clean)

    

homepage not found
{'First Name': 'Bogdan', 'Last Name': 'Oancea', 'RePEc Short-ID': 'poa5', 'Twitter': '@bogdan__oancea', 'Aff_Location0': ' Bucureşti, Romania', 'Aff_Location1': ' Ostrava, Czech Republic', 'Aff_Department0': '(90%) Facultatea de Administraţie şi Afaceri', 'Aff_Organisation0': 'Universitatea din Bucureşti', 'Aff_Department1': '(10%) Ekonomická fakulta', 'Aff_Organisation1': 'Vysoká Škola Báňská-Technická Univerzita Ostrava'}


In [224]:
df = pd.DataFrame.from_records([per_clean]).reindex(sorted(df.columns), axis=1)

In [225]:
df

,Aff_Department0,Aff_Department1,Aff_Location0,Aff_Location1,Aff_Organisation0,Aff_Organisation1,First Name,Last Name,RePEc Short-ID,Twitter
0,(90%) Facultatea de Administraţie şi Afaceri,(10%) Ekonomická fakulta,"Bucureşti, Romania","Ostrava, Czech Republic",Universitatea din Bucureşti,Vysoká Škola Báňská-Technická Univerzita Ostrava,Bogdan,Oancea,poa5,@bogdan__oancea
